# Using TensorFlow like Numpy

`tf.Tensor` very similar to Numpy `np.ndarray`
We create a tensor using `tf.constant()`

In [36]:
import tensorflow as tf
import numpy as np

# tf.Tensor

* **[`tf.Tensor`](https://www.tensorflow.org/api_docs/python/tf/Tensor) package**

### Creating a tf.Tensor

In [20]:
# Creating a tensor representing a matrix with 2 rows and 3 columns of floats:
tf.constant([[1., 2., 3.], [4., 5., 6.]])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [22]:
# Creating a tensor representing a scalar (simple value):
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

### tf.Tensor properties

In [24]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
print(t.shape)
print(t.dtype)

(2, 3)
<dtype: 'float32'>


### tf.Tensor indexing (like with Numpy)

In [26]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [29]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

### tf.Tensor operations

In [33]:
t + 10  # same as tf.add(t, 10)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [31]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [34]:
t @ tf.transpose(t)  # @ operator added in Python 3.5 for matrix multiplication, same as tf.matmul()

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

### Basic math operations

* `tf.add()`, `tf.multiply()`, `tf.square()`, `tf.exp()`, `tf.sqrt()`, etc.
* `tf.reshape()`, `tf.squeeze()`, `tf.tile()`
* `tf.reduce_mean()`, `tf.reduce_sum()`, `tf.reduce_max()`, `tf.math.log()` (same as `np.mean()`, `np.sum()`, etc.)
  * when the name differs from Numpy, there is a good reason:
    * `tf.transpose(t)`: new tensor created with its own copy of the transposed data   
      `t.T` attribute (NumPy): just a transposed view of the same data   
    * `tf.reduce_sum()` GPU kernel uses a reduce algorithm that does not guarantee order in which elements are added

### `tf.Tensor` and `np.ndarray`

* can create a tensor from ndarray and vice versa
* can apply TF operations to ndarray and NumPy operations to tensors
* **Important:**  
  NumPy uses **64-bit precision** by default,   
  TF uses **32-bit** by default (more than enough for NNs, runs faster, uses less RAM)   
  -> when you create a tensor from an ndarray, **set `dtype=tf.float32`**

In [37]:
a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [38]:
t.numpy()  # same as np.array(t)

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [39]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [40]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

### Type conversions

* can hurt performance, can go unnoticed
* no automatic type conversions in TF
* raises an exception if operations on **incompatible types** or **different type precision**

In [41]:
# adding float tensor and integer tensor
#   -> InvalidArgumentError...expected to be a float...
tf.constant(2.) + tf.constant(40)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]

In [44]:
# adding 32-bit float and 64-bit float
#   -> InvalidArgumentError...expected to be a double
tf.constant(2.0) + tf.cast(40, dtype=tf.float64)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2]

# tf.Variable

* **[`tf.Variable`](https://www.tensorflow.org/api_docs/python/tf/Variable) package**  

* `tf.Tensor` immutable  
  `tf.Variable` used for weights in NNs (backpropagation...), etc.
* Same usage as tensors (operations, plays nicely with NumPy, type conversions...)

In [47]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

* `assign()`, `assign_add()` (increment), `assign_sub()` (decrement by the given value)
* slicing with `assign()`, `scatter_update()` and `scatter_nd_update()`

In [48]:
v.assign(2 * v)                                 # [[2., 4.,  6.], [8., 10., 12.]]
v[0, 1].assign(42)                              # [[2., 42., 6.], [8., 10., 12.]]
v[:, 2].assign([0., 1.])                        # [[2., 42., 0.], [8., 10., 1.]]
v.scatter_nd_update(indices=[[0,0], [1,2]],     # [[100., 42., 0.], [8., 10., 200.]]
                    updates=[100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

# tf.SparseTensor

* **[`tf.sparse`](https://www.tensorflow.org/api_docs/python/tf/sparse) package**: operations for sparse tensors
* contain mostly zeros
* not as many operations as dense tensors
* `tf.sparse.reorder()`

In [88]:
s = tf.SparseTensor(indices=[[0,1], [1,2], [2,3]],  # indices of non-zero elements, in reading order
                    values=[1., 2, 3.],             # values of non-zero elements
                    dense_shape=[3, 4])
print(s)
tf.sparse.to_dense(s)

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 2]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))


<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 1., 0., 0.],
       [0., 0., 2., 0.],
       [0., 0., 0., 3.]], dtype=float32)>

# tf.TensorArray

* **[`tf.TensorArray`](https://www.tensorflow.org/api_docs/python/tf/TensorArray) package**
* lists of tensors
* all same shape and data type
* fixed size by default: `size=3`,   
  but can be made dynamic: `dynamic_size=True` (hinders performance)
* all elements must have same shape as first element inserted

In [96]:
array = tf.TensorArray(dtype=tf.float32, size=3)

array = array.write(0, tf.constant([1., 2.]))
array = array.write(1, tf.constant([3., 10.]))
array = array.write(2, tf.constant([5., 7.]))

print(array.read(1))   # returns and pops !! (replaces with same shape tensor of zeros)

print(array.stack())   # stack all elements into a regular tensor

tf.Tensor([ 3. 10.], shape=(2,), dtype=float32)
tf.Tensor(
[[1. 2.]
 [0. 0.]
 [5. 7.]], shape=(3, 2), dtype=float32)


# tf.RaggedTensor

* **[`tf.ragged`](https://www.tensorflow.org/api_docs/python/tf/ragged) package**: operations for ragged tensors
* static lists of lists of tensors   
  list of arrays of different sizes   
  tensor with one or more *ragged dimensions* (whose slices may have different lengths)
* each element of a ragged tensor is a **regular tensor**
* every tensor the same shape and data type

In [77]:
p = tf.constant(['café', 'coffee', 'caffè'])
r = tf.strings.unicode_decode(p, "UTF-8")
print(r)    # ragged tensor
print(r[1]) # regular tensor

<tf.RaggedTensor [[99, 97, 102, 233], [99, 111, 102, 102, 101, 101], [99, 97, 102, 102, 232]]>
tf.Tensor([ 99 111 102 102 101 101], shape=(6,), dtype=int32)


#### concatenate

In [82]:
# create a ragged tensor
r2 = tf.ragged.constant([[65, 66], [], [67]])

# concatenate along axis 0:
#  tensors in r2 added after tensors in r
print(tf.concat([r, r2], axis=0))

# concatenate along axis 1
#  i-th tensor in r3 concatenated with i-th tensor in r
r3 = tf.ragged.constant([[68, 69, 70], [71], [72, 73]])
print(tf.concat([r, r3], axis=1))

<tf.RaggedTensor [[99, 97, 102, 233], [99, 111, 102, 102, 101, 101], [99, 97, 102, 102, 232], [65, 66], [], [67]]>
<tf.RaggedTensor [[99, 97, 102, 233, 68, 69, 70], [99, 111, 102, 102, 101, 101, 71], [99, 97, 102, 102, 232, 72, 73]]>


####  `to_tensor()`

* convert to regular tensor
* padding shorter ones with zeros or set `default_value` to get them all equal lengths

In [84]:
r.to_tensor(default_value=-1)

<tf.Tensor: shape=(3, 6), dtype=int32, numpy=
array([[ 99,  97, 102, 233,  -1,  -1],
       [ 99, 111, 102, 102, 101, 101],
       [ 99,  97, 102, 102, 232,  -1]], dtype=int32)>

# String tensors

* **[`tf.strings`](https://www.tensorflow.org/api_docs/python/tf/strings) package**: operations for byte and Unicode strings (conversions...)
* regular tensors of type `tf.string`
* byte strings, not Unicode
* `tf.string` is **atomic**: its length does not appear in the tensor's shape,   
  must be converted to Unicode tensor (of type `tf.int32`), then length appears in shape
* `length()`: count number of bytes in a byte string,   
  `length(unit='UTF8_CHAR')`: number of code points in Unicode string

In [49]:
tf.constant(b'hello world')

<tf.Tensor: shape=(), dtype=string, numpy=b'hello world'>

In [50]:
tf.constant('café')
# tensor with Unicode string -> TF encodes it to utf-8

<tf.Tensor: shape=(), dtype=string, numpy=b'caf\xc3\xa9'>

In [51]:
# creating a tensor with Unicode string: 
# create an array of 32-bit integers, each representing a single Unicode code point
tf.constant([ord(c) for c in 'café'])

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 99,  97, 102, 233], dtype=int32)>

In [58]:
# convert Unicode tensor (int32) to byte string tensor
u = tf.constant([ord(c) for c in 'café'])
b = tf.strings.unicode_encode(u, "UTF-8")
tf.strings.length(b, unit='UTF8_CHAR')

<tf.Tensor: shape=(), dtype=int32, numpy=4>

In [57]:
# convert byte string tensor to Unicode tensor (int32)
tf.strings.unicode_decode(b, "UTF-8")

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 99,  97, 102, 233], dtype=int32)>

#### tensors with multiple strings

In [66]:
n = tf.constant(['café', 'coffee', 'caffè'])
tf.strings.length(n, unit='UTF8_CHAR')

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([4, 6, 5], dtype=int32)>

In [65]:
tf.strings.unicode_decode(n, 'UTF-8')

<tf.RaggedTensor [[99, 97, 102, 233], [99, 111, 102, 102, 101, 101], [99, 97, 102, 102, 232]]>

# Sets

* **[`tf.sets`](https://www.tensorflow.org/api_docs/python/tf/sets) package**: operations on sets
* represented as regular tensors
* each set is represented by a vector in the tensor's last axis

In [ ]:
tf.constant([[1, 2], [3, 4]])
# represents 2 sets {1, 2} and {3, 4}

# Queues

* **[`tf.Queue`](https://www.tensorflow.org/api_docs/python/tf/queue)**
* Queues, priority queues, random shuffle operations, padding...